In [1]:
from src.Data import Data
from src.Camera import Camera
from src.data_association import *
from src.utils import *

import pandas as pd
import cv2
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go   

In [2]:
data = Data()
camera = Camera()

In [3]:
kernel_threshold = 10000
dumping_factor = 2
min_inliners = 8
num_iterations = 15

In [4]:
def triangulate_point(p1, p2, P1, P2):
    """
    Triangulate a 3D point from two views.

    Parameters:
    p1 (array): The 2D projected point in the first image [x1, y1].
    p2 (array): The 2D projected point in the second image [x2, y2].
    P1 (array): The 3x4 camera projection matrix for the first image.
    P2 (array): The 3x4 camera projection matrix for the second image.

    Returns:
    array: The 3D coordinates of the triangulated point.
    """

    A = np.zeros((4, 4))
    A[0] = p1[0] * P1[2] - P1[0]
    A[1] = p1[1] * P1[2] - P1[1]
    A[2] = p2[0] * P2[2] - P2[0]
    A[3] = p2[1] * P2[2] - P2[1]

    U, S, Vt = np.linalg.svd(A)
    X = Vt[-1]  
    
    return X[:-1] / X[-1]

In [5]:
def triangulate_points(points_1, points_2, poses_1, poses_2):
    """
    Triangulate 3D points from two sets of 2D points.

    Parameters:
    points_1 (array): The 2D projected points in the first image.
    points_2 (array): The 2D projected points in the second image.
    T_0 (array): The 3x4 camera projection matrix for the first image.
    T_1 (array): The 3x4 camera projection matrix for the second image.

    Returns:
    array: The 3D coordinates of the triangulated points.
    """

    K = camera.get_camera_matrix()

    # T = np.linalg.inv(w_T_c1) @ w_T_c0
    # R = T[:3, :3] 
    # t = T[:3, 3].reshape(-1, 1)   

    # P_0 = K @ np.hstack((np.eye(3), np.zeros((3, 1))))
    # P_1 = K @ np.hstack((R, t))    

    # points_3d = []
    # for i, (p1, p2) in enumerate(zip(points_1, points_2)):
    #     point_3d = triangulate_point(p1, p2, P_0, P_1)
    #     if point_3d[2] > 0: points_3d.append(point_3d)
    # points_3d = np.array(points_3d)

    # return points_3d
    
    points_3d = []
    for i in range(len(points_1)):
        points_1[i] = points_1[i]
        points_2[i] = points_2[i]
        pose_1 = poses_1[i]
        pose_2 = poses_2[i]

        T = np.linalg.inv(pose_2) @ pose_1
        R = T[:3, :3]
        t = T[:3, 3].reshape(-1, 1)
        P_0 = K @ np.hstack((np.eye(3), np.zeros((3, 1))))
        P_1 = K @ np.hstack((R, t))

        point_3d = triangulate_point(points_1[i], points_2[i], P_0, P_1)
        if point_3d[2] > 0: points_3d.append(point_3d)

    return np.array(points_3d)

In [6]:
def data_association_on_appearance(set_1, set_2):
    points_1 = set_1['position']
    appearance_1 = set_1['appearance']

    points_2 = set_2['position']
    appearance_2 = set_2['appearance']

    matches = {'points_1':[], 'points_2':[], 'appearance':[]}

    for i in range(len(points_1)):
        for j in range(len(points_2)):
            if appearance_1[i] == appearance_2[j]:
                matches['points_1'].append(points_1[i])
                matches['points_2'].append(points_2[j])
                matches['appearance'].append(appearance_1[i])

    return matches

In [7]:
def projective_ICP(current_pose, image_points, map):

    matches = data_association_on_appearance(image_points, map)
    print('Num of matches:', len(matches['points_1']))
    reference_image_points = np.array(matches['points_1'])
    current_world_points = np.array(matches['points_2'])
    for i in range(num_iterations):

        w_T_c1, stop = one_step(current_pose, reference_image_points, current_world_points)
        if w_T_c1 is None: return current_pose

        current_pose = w_T_c1
        camera.set_c_T_w(np.linalg.inv(current_pose))

        if stop: break

    return current_pose

def one_step(current_pose, reference_image_points, current_world_points):

        if (len(current_world_points) == 0): return current_pose, True

        H, b, num_inliers, chi_inliers, chi_outliers, error = linearize(current_pose, reference_image_points, current_world_points)
        if num_inliers < min_inliners: return None, None

        H += np.eye(6) * dumping_factor
        dx = np.linalg.solve(H, -b)

        initial_guess = T2v(current_pose)
        updated_guess = initial_guess + dx
        w_T_c1 = v2T(updated_guess)

        return w_T_c1, error<0.1 or np.linalg.norm(dx)<1e-5

def linearize(current_pose, reference_image_points, currrent_world_points):
    H = np.zeros((6, 6))
    b = np.zeros(6)
    num_inliers = 0
    chi_inliers = 0
    chi_outliers = 0

    error = 0

    for i in range(len(reference_image_points)):
        reference_image_point = reference_image_points[i]
        current_world_point = currrent_world_points[i]

        e, jacobian = error_and_jacobian(current_pose, reference_image_point, current_world_point)
        if e is None or jacobian is None: continue

        chi = e.T @ e 
        
        lambda_ = 1.0
        is_inlier = True
        if chi > kernel_threshold:
            lambda_ = np.sqrt(kernel_threshold / chi)
            is_inlier = False
            chi_outliers += chi
        else:
            num_inliers += 1
            chi_inliers += chi

        if is_inlier:
            H += jacobian.T @ jacobian * lambda_
            b += jacobian.T @ e * lambda_

        error += np.linalg.norm(e)
    
    error /= len(reference_image_points)

    return H, b, num_inliers, chi_inliers, chi_outliers, error

def error_and_jacobian(current_pose, reference_image_point, current_world_point):

    is_inside, projected_image_point = camera.project_point(current_world_point)    
    if not is_inside: return None, None

    e = reference_image_point - projected_image_point

    p_hat_hom = np.linalg.inv(current_pose) @ np.append(current_world_point, 1)
    p_hat = p_hat_hom[:3] / p_hat_hom[3]
    p_hat_cam = camera.get_camera_matrix() @ p_hat

    J_icp = np.zeros((3, 6))
    J_icp[:3, :3] = np.eye(3)
    J_icp[:3, 3:] = -skew(p_hat)
    
    z_inv = 1.0 / p_hat_cam[2]
    z_inv2 = z_inv * z_inv

    J_proj = np.array([[z_inv, 0, -p_hat_cam[0] * z_inv2],
                        [0, z_inv, -p_hat_cam[1] * z_inv2]])
    
    J = J_proj @ camera.get_camera_matrix() @ J_icp

    return e, J

In [8]:
current_pose = np.eye(4)
poses = []
map = {'position': [], 'appearance': []}
features_map = {
    'position_0': [],
    'position_i': [],
    'appearance': [],
    'num_observations': [],
    'pose_0': [],
    'pose_i': []
}

In [9]:
meas_0 = data.get_measurements_data_points(0)
meas_1 = data.get_measurements_data_points(1)

matches = data_association_on_appearance(meas_0, meas_1)
points_0 = np.array(matches['points_1'])
points_1 = np.array(matches['points_2'])

w_T_c0 = np.eye(4)
poses.append(w_T_c0)

#** Estimate the relative pose between the two frames
K = camera.get_camera_matrix()
E, mask = cv2.findEssentialMat(points_0, points_1, K, method=cv2.RANSAC, prob=0.999, threshold=1.0)
_, R, t, mask = cv2.recoverPose(E, points_0, points_1, K, mask=mask)
c0_T_c1 = Rt2T(R, -t)
next_pose = w_T_c0 @ c0_T_c1

print("Estimated relative pose: ")
print(np.round(next_pose, 2))

#** Triangulate points
poses_0 = np.array([w_T_c0 for _ in range(len(points_0))])
poses_1 = np.array([next_pose for _ in range(len(points_1))])
points_3D = triangulate_points(points_0, points_1, poses_0, poses_1)
next_map = {'position':points_3D, 'appearance':matches['appearance']}

#| Update the state
for i in range(len(next_map['position'])):
    map['position'].append(next_map['position'][i])
    map['appearance'].append(next_map['appearance'][i])

for i in range(len(matches['points_1'])):
    features_map['position_0'].append(matches['points_1'][i])
    features_map['position_i'].append(matches['points_2'][i])
    features_map['appearance'].append(matches['appearance'][i])
    features_map['num_observations'].append(2)
    features_map['pose_0'].append(w_T_c0)
    features_map['pose_i'].append(next_pose)

current_pose = next_pose
current_map = next_map
camera.set_c_T_w(np.linalg.inv(current_pose))
poses.append(current_pose)

first_triangulated_points = points_3D

#** Plot the 3D points
fig = go.Figure()
x_coords = first_triangulated_points[:, 0]  
y_coords = first_triangulated_points[:, 1]
z_coords = first_triangulated_points[:, 2]
fig.add_trace(go.Scatter3d(x=x_coords, y=y_coords, z=z_coords, mode='markers', marker=dict(size=2)))
fig.show()  

Estimated relative pose: 
[[ 1.  0.  0.  0.]
 [-0.  1.  0.  0.]
 [-0. -0.  1.  1.]
 [ 0.  0.  0.  1.]]


In [10]:
map_df = pd.DataFrame(map)
map_df

,position,appearance
0,"[14.558053702976437, -3.747522397379311, 12.96...","[-0.668052, -0.119791, 0.76015, 0.658402, -0.3..."
1,"[4.732030357857323, -3.7393348624614986, 6.927...","[-0.80072, 0.0616159, 0.514588, -0.39141, 0.98..."
2,"[-33.284972670395476, -8.497416814901483, 23.7...","[0.746543, 0.662075, 0.958868, 0.487622, 0.806..."
3,"[-1.0668033075424719, -9.414639364671393, 14.0...","[-0.439916, 0.0922137, 0.438537, -0.773439, -0..."
4,"[23.887833088769145, -2.98454160331673, 22.676...","[0.104633, 0.839182, 0.371973, 0.619571, 0.395..."
...,...,...
110,"[-6.8258135249797025, -3.460991658458284, 22.7...","[-0.426205, -0.741321, 0.567446, 0.887447, 0.2..."
111,"[-1.6045085145192004, -4.091317772070281, 16.1...","[-0.0250311, -0.80992, -0.385503, 0.413624, -0..."
112,"[1.7674309631850105, -3.6071067048272725, 15.5...","[0.683682, -0.00590533, 0.276129, -0.692478, 0..."
113,"[-9.659952027578813, -8.502692997679217, 8.281...","[0.0400809, 0.470501, -0.1505, 0.457585, 0.206..."


In [11]:
features_map_df = pd.DataFrame(features_map)
features_map_df

,position_0,position_i,appearance,num_observations,pose_0,pose_i
0,"[522.119, 187.968]","[539.005, 183.622]","[-0.668052, -0.119791, 0.76015, 0.658402, -0.3...",2,"[[1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [...","[[0.9999999998853839, 2.7520408189051637e-06, ..."
1,"[442.949, 142.838]","[463.689, 126.449]","[-0.80072, 0.0616159, 0.514588, -0.39141, 0.98...",2,"[[1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [...","[[0.9999999998853839, 2.7520408189051637e-06, ..."
2,"[67.7619, 175.604]","[56.6646, 172.771]","[0.746543, 0.662075, 0.958868, 0.487622, 0.806...",2,"[[1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [...","[[0.9999999998853839, 2.7520408189051637e-06, ..."
3,"[306.371, 119.738]","[305.329, 110.546]","[-0.439916, 0.0922137, 0.438537, -0.773439, -0...",2,"[[1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [...","[[0.9999999998853839, 2.7520408189051637e-06, ..."
4,"[509.615, 216.308]","[518.356, 215.216]","[0.104633, 0.839182, 0.371973, 0.619571, 0.395...",2,"[[1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [...","[[0.9999999998853839, 2.7520408189051637e-06, ..."
...,...,...,...,...,...,...
110,"[265.905, 212.571]","[263.409, 211.305]","[-0.426205, -0.741321, 0.567446, 0.887447, 0.2...",2,"[[1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [...","[[0.9999999998853839, 2.7520408189051637e-06, ..."
111,"[302.141, 194.465]","[300.962, 191.459]","[-0.0250311, -0.80992, -0.385503, 0.413624, -0...",2,"[[1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [...","[[0.9999999998853839, 2.7520408189051637e-06, ..."
112,"[340.454, 198.249]","[341.861, 195.379]","[0.683682, -0.00590533, 0.276129, -0.692478, 0...",2,"[[1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [...","[[0.9999999998853839, 2.7520408189051637e-06, ..."
113,"[110.052, 55.203]","[81.2067, 29.8137]","[0.0400809, 0.470501, -0.1505, 0.457585, 0.206...",2,"[[1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [...","[[0.9999999998853839, 2.7520408189051637e-06, ..."


In [12]:
for i in range(2,20):

    prev_meas = data.get_measurements_data_points(i-1)
    curr_meas = data.get_measurements_data_points(i)

    next_pose = projective_ICP(current_pose, curr_meas, map)
    print("Estimated pose: ")
    print(np.round(next_pose, 2))

    matches = data_association_on_appearance(prev_meas, curr_meas)
    points_1 = np.array(matches['points_1'])
    points_2 = np.array(matches['points_2'])

    for i in range(len(matches['points_1'])):
        point_1 = matches['points_1'][i]
        point_2 = matches['points_2'][i]
        appearance = matches['appearance'][i]
        if appearance in features_map['appearance']:
            index = features_map['appearance'].index(appearance)
            features_map['position_i'][index] = point_2
            features_map['num_observations'][index] += 1
            features_map['pose_i'][index] = next_pose
        else:
            features_map['position_0'].append(point_1)
            features_map['position_i'].append(point_2)
            features_map['appearance'].append(appearance)
            features_map['num_observations'].append(2)
            features_map['pose_0'].append(current_pose)
            features_map['pose_i'].append(next_pose)

    points_1 = []
    points_2 = []
    appearances = []
    poses_1 = []
    poses_2 = []
    for i in range(len(features_map['appearance'])):
        if features_map['num_observations'][i] > 5:# and features_map['appearance'][i] not in map['appearance']:
            points_1.append(features_map['position_0'][i])
            points_2.append(features_map['position_i'][i])
            appearances.append(features_map['appearance'][i])
            poses_1.append(features_map['pose_0'][i])
            poses_2.append(features_map['pose_i'][i])
    points_1 = np.array(points_1)
    points_2 = np.array(points_2)
    poses_1 = np.array(poses_1)
    poses_2 = np.array(poses_2)
    print("Number of candidate points for triangulation: ", len(points_1))

    points_3D = triangulate_points(points_1, points_2, poses_1, poses_2)
    next_map = {'position':points_3D, 'appearance':appearances}
    print("Number of triangulated points: ", len(points_3D))
    
    for i in range(len(next_map['position'])):
        if next_map['appearance'][i] not in map['appearance']:
            map['position'].append(next_map['position'][i])
            map['appearance'].append(next_map['appearance'][i])
    print('Num of points in map: ', len(map['position']))

    #** Update the state
    current_pose = next_pose
    current_map = next_map
    camera.set_c_T_w(np.linalg.inv(current_pose))
    poses.append(current_pose)


    last_trianguled_points = points_3D

    fig = go.Figure()
    x_coords = []  
    y_coords = []
    z_coords = []
    for i in range(len(map['position'])):
        x_coords.append(map['position'][i][0])
        y_coords.append(map['position'][i][1])
        z_coords.append(map['position'][i][2])
    fig.add_trace(go.Scatter3d(x=x_coords, y=y_coords, z=z_coords, mode='markers', marker=dict(size=4)))

    x_coord_first = first_triangulated_points[:, 0]
    y_coord_first = first_triangulated_points[:, 1]
    z_coord_first = first_triangulated_points[:, 2]
    fig.add_trace(go.Scatter3d(x=x_coord_first, y=y_coord_first, z=z_coord_first, mode='markers', marker=dict(size=2, color='red')))

    if len(last_trianguled_points) > 0:
        x_coord_last = last_trianguled_points[:, 0] 
        y_coord_last = last_trianguled_points[:, 1]
        z_coord_last = last_trianguled_points[:, 2]
        fig.add_trace(go.Scatter3d(x=x_coord_last, y=y_coord_last, z=z_coord_last, mode='markers', marker=dict(size=2, color='yellow')))

    fig.show()  
    print()

Num of matches: 108
Estimated pose: 
[[ 1.  0.  0.  0.]
 [-0.  1.  0.  0.]
 [-0. -0.  1.  2.]
 [ 0.  0.  0.  1.]]
Number of triangulated points:  117
Num of points in map:  124



Num of matches: 112
Estimated pose: 
[[ 1.   -0.    0.   -0.02]
 [ 0.    1.   -0.    0.02]
 [-0.    0.    1.    2.99]
 [ 0.    0.    0.    1.  ]]
Number of triangulated points:  118
Num of points in map:  130



Num of matches: 99
Estimated pose: 
[[ 1.   -0.    0.   -0.04]
 [ 0.    1.   -0.    0.05]
 [-0.    0.    1.    4.  ]
 [ 0.    0.    0.    1.  ]]
Number of triangulated points:  104
Num of points in map:  135



Num of matches: 91
Estimated pose: 
[[ 1.   -0.    0.01 -0.08]
 [ 0.    1.   -0.01  0.18]
 [-0.01  0.01  1.    4.95]
 [ 0.    0.    0.    1.  ]]
Number of triangulated points:  96
Num of points in map:  140



Num of matches: 89
Estimated pose: 
[[ 1.   -0.    0.   -0.09]
 [ 0.    1.   -0.02  0.28]
 [-0.    0.02  1.    5.92]
 [ 0.    0.    0.    1.  ]]
Number of triangulated points:  96
Num of points in map:  147



Num of matches: 92
Estimated pose: 
[[ 1.   -0.    0.   -0.18]
 [ 0.    1.   -0.02  0.32]
 [-0.    0.02  1.    6.88]
 [ 0.    0.    0.    1.  ]]
Number of triangulated points:  97
Num of points in map:  152



Num of matches: 85
Estimated pose: 
[[ 1.   -0.   -0.01 -0.  ]
 [ 0.    1.   -0.05  0.57]
 [ 0.01  0.05  1.    7.84]
 [ 0.    0.    0.    1.  ]]
Number of triangulated points:  90
Num of points in map:  157



Num of matches: 77
Estimated pose: 
[[ 1.   -0.01 -0.01 -0.22]
 [ 0.01  1.   -0.04  0.54]
 [ 0.01  0.04  1.    8.33]
 [ 0.    0.    0.    1.  ]]
Number of triangulated points:  85
Num of points in map:  170


In [13]:
map_df = pd.DataFrame(map)  
map_df

,position,appearance
0,"[14.558053702976437, -3.747522397379311, 12.96...","[-0.668052, -0.119791, 0.76015, 0.658402, -0.3..."
1,"[4.732030357857323, -3.7393348624614986, 6.927...","[-0.80072, 0.0616159, 0.514588, -0.39141, 0.98..."
2,"[-33.284972670395476, -8.497416814901483, 23.7...","[0.746543, 0.662075, 0.958868, 0.487622, 0.806..."
3,"[-1.0668033075424719, -9.414639364671393, 14.0...","[-0.439916, 0.0922137, 0.438537, -0.773439, -0..."
4,"[23.887833088769145, -2.98454160331673, 22.676...","[0.104633, 0.839182, 0.371973, 0.619571, 0.395..."
...,...,...
165,"[-4.288314709828009, -6.176759323870499, 8.741...","[-0.130899, -0.639431, -0.791487, 0.728055, -0..."
166,"[18.440576321285196, -0.2096962859045931, 23.0...","[-0.25091, -0.909769, -0.957132, -0.276385, -0..."
167,"[-1.1619293336538237, -2.1902651300615052, 5.1...","[-0.103442, 0.446212, 0.705412, 0.109605, 0.35..."
168,"[-4.008682886917105, -2.5676621479506228, 4.93...","[-0.779044, -0.881711, 0.747622, 0.545106, -0...."


In [14]:
features_map_df = pd.DataFrame(features_map)
features_map_df

,position_0,position_i,appearance,num_observations,pose_0,pose_i
0,"[522.119, 187.968]","[612.247, 164.767]","[-0.668052, -0.119791, 0.76015, 0.658402, -0.3...",5,"[[1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [...","[[0.9999971985816956, -0.0004913200973533094, ..."
1,"[442.949, 142.838]","[610.895, 10.1166]","[-0.80072, 0.0616159, 0.514588, -0.39141, 0.98...",5,"[[1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [...","[[0.9999971985816956, -0.0004913200973533094, ..."
2,"[67.7619, 175.604]","[0.398867, 158.406]","[0.746543, 0.662075, 0.958868, 0.487622, 0.806...",6,"[[1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [...","[[0.9999802708625793, -0.001047504716552794, 0..."
3,"[306.371, 119.738]","[292.909, 0.948853]","[-0.439916, 0.0922137, 0.438537, -0.773439, -0...",8,"[[1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [...","[[0.9999778866767883, -0.004391441587358713, 0..."
4,"[509.615, 216.308]","[634.132, 200.75]","[0.104633, 0.839182, 0.371973, 0.619571, 0.395...",10,"[[1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [...","[[0.9998818635940552, -0.010354516096413136, -..."
...,...,...,...,...,...,...
166,"[251.267, 186.94]","[248.295, 184.645]","[-0.25091, -0.909769, -0.957132, -0.276385, -0...",2,"[[0.9999724626541138, -0.0016809485387057066, ...","[[0.9998818635940552, -0.010354516096413136, -..."
167,"[27.4223, 235.854]","[14.9872, 235.678]","[-0.103442, 0.446212, 0.705412, 0.109605, 0.35...",2,"[[0.9999724626541138, -0.0016809485387057066, ...","[[0.9998818635940552, -0.010354516096413136, -..."
168,"[463.836, 236.527]","[469.838, 236.382]","[-0.779044, -0.881711, 0.747622, 0.545106, -0....",2,"[[0.9999724626541138, -0.0016809485387057066, ...","[[0.9998818635940552, -0.010354516096413136, -..."
169,"[537.262, 194.566]","[546.35, 192.666]","[-0.439323, -0.177775, -0.453454, 0.571201, 0....",2,"[[0.9999724626541138, -0.0016809485387057066, ...","[[0.9998818635940552, -0.010354516096413136, -..."


In [15]:
x_est = []
y_est = []
z_est = []

x_gt = []
y_gt = []
z_gt = []

gt_poses = data.get_trajectory_data()
for i in range(len(poses)):
    C = camera.get_camera_transform()
    pose_i = poses[i]
    pose_i = C @ pose_i
    t_est = pose_i[:3, 3]
    x_est.append(t_est[0]*0.20)
    y_est.append(t_est[1]*0.20)
    z_est.append(t_est[2]*0.20)

    x_gt.append(gt_poses[i][0])
    y_gt.append(gt_poses[i][1])
    z_gt.append(0)

print("num of estimated poses: ", len(x_est))
print("num of ground truth poses: ", len(x_gt))

fig = go.Figure()
fig.update_layout(scene = dict(
                    #xaxis = dict(nticks=4, range=[-2,5],),
                    yaxis = dict(nticks=4, range=[-2,2],),
                    zaxis = dict(nticks=4, range=[-1,1],),))

fig.add_trace(go.Scatter3d(x=x_est, y=y_est, z=z_est, mode='lines+markers', marker=dict(size=2, color='red')))
fig.add_trace(go.Scatter3d(x=x_gt, y=y_gt, z=z_gt, mode='lines+markers', marker=dict(size=2, color='green')))
fig.show()

num of estimated poses:  10
num of ground truth poses:  10


In [16]:
fig = go.Figure()
x_coords = []  
y_coords = []
z_coords = []
for i in range(len(map['position'])):
    x_coords.append(map['position'][i][0])
    y_coords.append(map['position'][i][1])
    z_coords.append(map['position'][i][2])

fig.add_trace(go.Scatter3d(x=x_coords, y=y_coords, z=z_coords, mode='markers', marker=dict(size=2)))
fig.show()  